In [19]:
from sklearn.datasets import fetch_openml
db = fetch_openml('Fashion-MNIST', version=1)

c:\Users\mathn\.conda\envs\Torch_PY38\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [20]:
# 과제 목표 : 다중 분류 + 검증 데이터셋 사용(no_grad)
feature = db.data # 피쳐 설정
target = db.target # 타겟 설정 

In [21]:
from sklearn.model_selection import train_test_split
# 테스트셋 분리 : x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.2)

In [22]:
from sklearn.preprocessing import StandardScaler
# 표준화 스케일링
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [23]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
# 다중 분류이므로, 원핫인코딩 사용(타겟 데이터 간 상관관계가 없으므로 LabelEncoder 사용하지 않음)
encoder = OneHotEncoder()
# 1차원인 y_train, y_test를 2차원으로 변환 후, 원핫인코딩
y_train = encoder.fit_transform(np.array(y_train.values).reshape(-1,1)).toarray()
y_test = encoder.transform(np.array(y_test.values).reshape(-1,1)).toarray()

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader # 내일 배울 거 예습! 

# 연산을 하기 위해서, 모든 텐서를 float텐서로 변환
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

In [25]:
# 훈련용 TensorDataset과 DataLoader 생성
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# no_grad를 사용하기 위해, 테스트용 TensorDataset과 DataLoader 생성
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [26]:
class Model(nn.Module): # 오늘 배운 거! 
    def __init__(self):
        super().__init__() # nn.Module 상속
        self.fc1 = nn.Linear(784, 256) # 28 X 28 = 784 이므로, 들어가는 피쳐의 값은 784개임
        self.fc2 = nn.Linear(256, 128) # 은닉층1
        self.fc3 = nn.Linear(128, 64) # 은닉층2 
        self.fc4 = nn.Linear(64, 10) # 10개의 클래스로 분류해야 하므로, 10개의 피쳐가 나오도록 설정
        
    def forward(self, x):  # 오늘 배운 거2!
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
model = Model() # 모델 객체 생성
# 이제 옵티마이저와 손실함수를 설정해야한다! 

In [27]:
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adam 옵티마이저 사용 : 오늘 배운 거3!
loss_function = nn.CrossEntropyLoss()
epochs = 10
for e in range(epochs):
    for inputs, labels in train_loader: # 데이터를 하나씩 꺼내오자...
        
        outputs = model(inputs) # x_train을 모델에 넣어서 예측값을 구함
        
        loss = loss_function(outputs, torch.max(labels, 1)[1]) # 손실함수 정의 
        # torch.max(labels, 1)[1] : dim=1, 즉 행을 기준으로 최댓값의 인덱스를 반환
        #=================================================
        optimizer.zero_grad() 
        loss.backward()                      # 늘 하던 파트
        optimizer.step()
        #=================================================

In [28]:
with torch.no_grad(): # 검증시간, 모든 업데이트를 멈춘다! 
    total = y_test.shape[0]
    outputs = model(x_test) # 예측값 생성
    _, predicted = torch.max(outputs, 1) # 아까와 똑같다, 인덱스를 갖고옴, 근데 원핫인코딩 좀 별로다..일일히 변환해야 함;;
    correct = (predicted == torch.max(y_test, 1)[1]).sum().item() # 짜다보니 이렇게 됐다...
    print(f"Accuracy: {correct/total}") # 정확도 출력 

Accuracy: 0.8895


In [29]:
# 88.9%!  생각보다 잘나왔다~